In [ ]:
import apimoex
import requests

In [53]:
dict_comp = {
    'SBERBANK': ['SBER', 'SBERP', 'SBER03'],       
    'GAZPROM': ['GAZP', 'GAZP-RM', 'GAZP03'],      
    'SEVERSTAL': ['CHMF', 'CHMF-RM']               
}

In [54]:
def get_minute_data(ticker_name, start, end):
    url = f"https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/{ticker_name}/candles.json"
    all_data = pd.DataFrame()
    current_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)
    
    with requests.Session() as session:
        while current_date <= end_date:
            params = {
                'from': current_date.strftime('%Y-%m-%d'),
                'till': current_date.strftime('%Y-%m-%d'),
                'interval': 1  # Интервал в 1 минуту
            }
            
            try:
                response = session.get(url, params=params)
                data = response.json()
                
                if 'candles' in data and 'data' in data['candles']:
                    candles_data = data['candles']['data']
                    columns = data['candles']['columns']
                    daily_data = pd.DataFrame(candles_data, columns=columns)
                    
                    if not daily_data.empty:
                        daily_data['begin'] = pd.to_datetime(daily_data['begin'])
                        daily_data.set_index('begin', inplace=True)
                        all_data = pd.concat([all_data, daily_data])
                else:
                    print(f"No data for {ticker_name} on {current_date.strftime('%Y-%m-%d')}")
                    
            except Exception as e:
                print(f"Error fetching data for {ticker_name} on {current_date.strftime('%Y-%m-%d')}: {e}")
            
            current_date += timedelta(days=1)
            
    return all_data

In [55]:
all_shares = pd.DataFrame()

for company, tickers in dict_comp.items():
    company_data = pd.DataFrame()
    
    for ticker_name in tickers:
        try:
            share = get_minute_data(ticker_name, start='2023-01-01', end='2023-07-01')
            if not share.empty:
                share = share.add_prefix(f"{company}_")
                company_data = pd.concat([company_data, share], axis=0)
                
        except Exception as e:
            print(f"Error fetching data for {ticker_name}: {e}")
            continue


    if not company_data.empty:
        company_data = company_data[~company_data.index.duplicated(keep='last')]
        company_data = company_data.sort_index().fillna(method='ffill')
        

        all_shares = pd.concat([all_shares, company_data], axis=1)



C:\Users\Ayazoro\AppData\Local\Temp\ipykernel_14876\1363011089.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  company_data = company_data.sort_index().fillna(method='ffill')
C:\Users\Ayazoro\AppData\Local\Temp\ipykernel_14876\1363011089.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  company_data = company_data.sort_index().fillna(method='ffill')
C:\Users\Ayazoro\AppData\Local\Temp\ipykernel_14876\1363011089.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  company_data = company_data.sort_index().fillna(method='ffill')


In [56]:
all_shares

,SBERBANK_open,SBERBANK_close,SBERBANK_high,SBERBANK_low,SBERBANK_value,SBERBANK_volume,SBERBANK_end,GAZPROM_open,GAZPROM_close,GAZPROM_high,...,GAZPROM_value,GAZPROM_volume,GAZPROM_end,SEVERSTAL_open,SEVERSTAL_close,SEVERSTAL_high,SEVERSTAL_low,SEVERSTAL_value,SEVERSTAL_volume,SEVERSTAL_end
begin,,,,,,,,,,,,,,,,,,,,,
2023-01-03 09:59:00,141.98,141.98,141.98,141.98,623292.2,4390.0,2023-01-03 09:59:59,163.00,163.00,163.00,...,3202950.0,19650.0,2023-01-03 09:59:59,901.4,901.4,901.4,901.4,544445.6,604.0,2023-01-03 09:59:59
2023-01-03 10:00:00,141.39,141.77,141.98,141.35,5222705.0,36870.0,2023-01-03 10:00:59,163.02,162.97,163.20,...,9785043.9,60010.0,2023-01-03 10:00:59,902.4,900.0,903.0,899.4,2977627.8,3304.0,2023-01-03 10:00:59
2023-01-03 10:01:00,141.75,141.66,141.99,141.47,2848250.1,20070.0,2023-01-03 10:01:59,162.91,162.51,163.02,...,4853519.7,29840.0,2023-01-03 10:01:59,900.0,900.2,900.6,899.4,1663076.8,1848.0,2023-01-03 10:01:59
2023-01-03 10:02:00,141.58,141.64,141.66,141.58,1172537.4,8280.0,2023-01-03 10:02:59,162.51,162.66,162.75,...,6973666.8,42910.0,2023-01-03 10:02:59,900.0,899.6,900.0,899.2,336374.6,374.0,2023-01-03 10:02:59
2023-01-03 10:03:00,141.64,141.70,141.74,141.64,508607.6,3590.0,2023-01-03 10:03:59,162.66,162.44,162.67,...,5747959.5,35360.0,2023-01-03 10:03:59,899.6,900.0,900.2,899.6,1099767.4,1222.0,2023-01-03 10:03:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30 18:16:00,237.76,237.77,237.78,237.76,106995.5,450.0,2023-06-30 18:16:59,167.05,167.04,167.05,...,355812.6,2130.0,2023-06-30 18:16:59,1180.2,1180.2,1180.2,1180.2,16522.8,14.0,2023-06-30 18:16:59
2023-06-30 18:17:00,237.77,237.77,237.77,237.77,199726.8,840.0,2023-06-30 18:17:59,167.05,167.00,167.10,...,12340706.5,73860.0,2023-06-30 18:17:59,1180.2,1180.2,1180.6,1180.0,694083.2,588.0,2023-06-30 18:17:59
2023-06-30 18:18:00,237.76,237.76,237.79,237.76,73710.8,310.0,2023-06-30 18:18:59,167.02,166.98,167.02,...,2271185.0,13600.0,2023-06-30 18:18:59,1180.4,1180.0,1180.4,1180.0,167560.8,142.0,2023-06-30 18:18:59


In [57]:
#all_shares.to_excel("Для_машинки.xlsx")  